<a href="https://colab.research.google.com/github/Jwaminju/docs/blob/main/2024%20OSSCA/Hugging_Face_2%EC%A3%BC%EC%B0%A8_%EB%AC%B8%EC%A0%9C_ipynb%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Hugging Face Transformers 실습하며 배우기 🤗

작성자: Hugging Face KREW, [김하림](https://github.com/harheem)

트랜스포머 기반의 사전 학습 모델의 뛰어난 점은 엄청난 양의 텍스트 데이터를 사용해 단어와 문장의 관계를 스스로 학습한다는 점이에요. 책, 뉴스, 웹 문서 등 다양한 텍스트를 분석하며 언어의 구조와 의미를 이해하게 된답니다.

하지만 이렇게 학습한 모델을 특정 작업에 활용하려면 추가 학습이 필요할 수 있어요. 이를 '파인튜닝'이라고 합니다. 파인튜닝은 적은 양의 labeled 데이터를 사용해 모델을 우리가 원하는 작업에 맞게 조정하는 과정을 의미해요.

이번 주에는 이러한 파인튜닝을 위한 데이터셋을 어떻게 준비하는지 배워보도록 할게요. 좋은 데이터셋을 만드는 것이 정확한 모델을 얻는 첫 걸음이니까요 🤗

### #문제 004

[IMDB 데이터세트](https://huggingface.co/datasets/stanfordnlp/imdb)는 영화 리뷰 코멘트의 긍정적/부정적 감성을 판단하기 위해 사용하는 감성 분석 데이터세트입니다. 데이터세트 구성은 다음과 같아요.
- 25,000개 학습 데이터 (텍스트 및 레이블)
- 25,000개 테스트 데이터 (텍스트 및 레이블)

이 50,000개 데이터세트를 다운로드하는 코드를 작성해보세요. 그리고 학습과 테스트를 위해 랜덤하게 1,000개씩 데이터를 추출해보세요.

💡 HINT

1.	필요한 라이브러리 설치:

  먼저, Hugging Face의 🤗 datasets 라이브러리를 설치해야 합니다. 커맨드 라인에서 다음 명령어를 실행하세요.

  `!pip install datasets`
2. `load_dataset()`을 이용하여 데이터세트 불러오세요.

  ```python
    from datasets import load_dataset
    dataset = load_dataset('여기에 데이터세트 이름을 넣어보세요')
  ```
  
3. datasets의 shuffle() 메소드와 select() 메소드를 사용하여 데이터를 추출해보세요.
  ```python
    dataset['train'].shuffle(seed=seed).select(range(n))
  ```

> 못 풀겠어도 걱정하지 마세요. [이 문서](https://huggingface.co/docs/datasets/main/en/loading)에서 답을 찾을 수 있어요 👀

In [3]:
# 필요한 라이브러리 불러오기
# !pip install datasets

# 데이터 불러오기
from datasets import load_dataset
dataset = load_dataset('imdb')
len(dataset['train'])

# 데이터 추출하기
train_dataset = dataset['train'].shuffle(seed=42).select(range(25000))
test_dataset = dataset['test'].shuffle(seed=42).select(range(25000))

In [4]:
# 학습 데이터셋에서 첫 번째 샘플 출력
print("\n학습 데이터 샘플:")
print(train_dataset[0])

# 테스트 데이터셋에서 첫 번째 샘플 출력
print("\n테스트 데이터 샘플:")
print(test_dataset[0])


학습 데이터 샘플:
{'text': 'There is no relation at all between Fortier and Profiler but the fact that both are police series about violent crimes. Profiler looks crispy, Fortier looks classic. Profiler plots are quite simple. Fortier\'s plot are far more complicated... Fortier looks more like Prime Suspect, if we have to spot similarities... The main character is weak and weirdo, but have "clairvoyance". People like to compare, to judge, to evaluate. How about just enjoying? Funny thing too, people writing Fortier looks American but, on the other hand, arguing they prefer American series (!!!). Maybe it\'s the language, or the spirit, but I think this series is more English than American. By the way, the actors are really good and funny. The acting is not superficial at all...', 'label': 1}

테스트 데이터 샘플:
{'text': "<br /><br />When I unsuspectedly rented A Thousand Acres, I thought I was in for an entertaining King Lear story and of course Michelle Pfeiffer was in it, so what could go wrong?<

### #문제 005

문제 005에서 negative `'0'`과 positive `'1'`로 인코딩된 1,000개의 학습 데이터 중에서 800개는 학습 `training` 용도로, 200개는 검증 `validation` 용도로 랜덤하게 나누어 보세요.


> 1,000개의 데이터를 800개의 학습용과 200개의 검증용으로 나누는 이유는 모델의 효과적인 학습과 평가를 위해서입니다. 학습 데이터로 모델을 훈련시키고, 검증 데이터로 모델의 성능을 평가합니다. 이렇게 분리함으로써 모델이 학습 데이터만을 '외우는' 과적합을 방지하고, 새로운 데이터에 대한 일반화 능력을 확인할 수 있습니다. 또한, 검증 결과를 바탕으로 모델의 하이퍼파라미터를 조정하여 성능을 개선할 수 있습니다. 이는 머신러닝 모델 개발에서 중요한 과정으로, 더 견고하고 신뢰할 수 있는 모델을 만드는 데 도움이 됩니다.

💡 HINT

1. `train_test_split()` 메소드를 사용하여 학습 데이터를 8:2로 나누어보세요.

2. `train_test_split()` 메소드에서 사용하는 [파라미터를 확인](https://huggingface.co/docs/datasets/v2.20.0/en/package_reference/main_classes#datasets.Dataset.train_test_split)하고 데이터를 나누는 다양한 방법에 대해 익혀보세요.

> 못 풀겠어도 걱정하지 마세요. [이 문서](https://huggingface.co/docs/datasets/process#split)에서 답을 찾을 수 있어요 👀

In [5]:
# 학습 데이터를 8:2 비율로 나누기
split_dataset = dataset['train'].train_test_split(train_size=0.8, test_size=0.2)

# split_dataset에서 훈련 데이터, 검증 데이터 가져오기
train_dataset = split_dataset['train']
valid_dataset = split_dataset['test']

# 결과 확인
print("학습 데이터 크기:", len(train_dataset))
print("검증 데이터 크기:", len(valid_dataset))

# 학습 데이터의 일부 샘플 출력
print("\n학습 데이터 샘플:")
print(train_dataset[:5])

# 검증 데이터의 일부 샘플 출력
print("\n검증 데이터 샘플:")
print(valid_dataset[:5])

학습 데이터 크기: 20000
검증 데이터 크기: 5000

학습 데이터 샘플:
{'text': ["As in Amelie, recent French films seem to be taking a stereotypical male-female relationship slant, centered on a female finding her one true love. In this case, desperation leads to a convict, which leads to her evolution into a mob prototype. Clever and surprising story in many ways, except that the female is there to support the male.<br /><br />For those of us that don't speak French, the subtitles are a little quick, but not unreasonable.<br /><br />The soundtrack, as seems to be increasingly the case with European films, is great and in perfect sync with the film's variations. Nothing seems forced. Visually, it reminds me of various urban horror movies. There's a Wes Craven in Chicago feel to it.", 'Of all the breakdancing / hip-hop films released between 1983 and 1986, the 1984 film Beat Street is unquestionably the best one. The story follows a DJ, his younger breakdancing brother, a graffiti artist and a wanna-be showbiz 

### #문제 006

문제 004와 005에서 추출한 학습 데이터(800개), 검증 데이터(200개), 테스트 데이터(1,000개)를 사전학습된 distilbert-base-uncased 모델에 투입하기 위해, 토크나이저를 사용해 인코딩하세요.

> 토크나이징, 인코딩은 텍스트를 기계가 이해할 수 있는 형태로 변환하는 과정입니다. 토큰은 텍스트의 의미 있는 가장 작은 단위로, 보통 단어나 부분 단어를 의미합니다. 토크나이징은 텍스트를 이러한 토큰들로 나누는 과정입니다. 예를 들어, "I love NLP"라는 문장은 "I", "love", "NLP"라는 토큰으로 나눌 수 있습니다.

> 인코딩은 나눈 토큰들을 숫자로 변환하는 과정입니다. 각 토큰에는 사전에 정의된 어휘 사전(vocabulary)에 따라 고유한 숫자(ID)가 할당됩니다. 예를 들어, "I"는 1021, "love"는 2293, "NLP"는 11234로 변환될 수 있습니다. 이렇게 변환된 숫자 시퀀스를 `input_ids`라고 부르며, 이는 [1021, 2293, 11234]와 같은 형태를 가집니다. 이렇게 변환된 숫자 형태의 데이터는 기계학습 모델이 처리하기에 적합한 형태가 됩니다.

💡 HINT

1. Hugging Face의 DistilBertTokenizerFast 사용하여 ['distilbert-base-uncased'](https://huggingface.co/distilbert/distilbert-base-uncased) 토크나이저를 불러오세요.
  ```python
    from transformers import DistilBertTokenizerFast
    tokenizer = DistilBertTokenizerFast.from_pretrained("불러올 토크나이저 이름을 입력하세요.")
  ```

2. 다음의 파라미터를 사용하여 데이터를 토크나이징해보세요. 토크나이저의 [`__call__` 메소드](https://huggingface.co/docs/transformers/main_classes/tokenizer#transformers.PreTrainedTokenizer.__call__)를 직접 사용하여 텍스트를 토크나이징할 수 있어요.
  - `padding=True`: 모든 시퀀스의 길이를 동일하게 맞춥니다.
  - `truncation=True`: 최대 길이를 초과하는 시퀀스를 잘라냅니다.
  - `max_length=512`: 최대 시퀀스 길이를 512로 설정합니다.
  - `return_tensors="pt"`: PyTorch 텐서 형태로 결과를 반환합니다.


In [6]:
# DistilBERT Fast 토크나이저 로드
from transformers import DistilBertTokenizerFast
tokenizer = DistilBertTokenizerFast.from_pretrained("distilbert-base-uncased")

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

In [7]:
# 데이터 토크나이징하기
tokenized_train = tokenizer(train_dataset['text'], padding=True, truncation=True, max_length=512, return_tensors="pt")
tokenized_val = tokenizer(valid_dataset['text'], padding=True, truncation=True, max_length=512, return_tensors="pt")
tokenized_test = tokenizer(test_dataset['text'], padding=True, truncation=True, max_length=512, return_tensors="pt")

In [8]:
# 결과 확인
print("학습 데이터 인풋 크기:", len(tokenized_train['input_ids']))
print("검증 데이터 인풋 크기:", len(tokenized_val['input_ids']))
print("테스트 데이터 인풋 크기:", len(tokenized_test['input_ids']))

# 0번째 텍스트의 5번째 토큰까지의 input_ids 출력
print("\n토크나이징된 학습 데이터 샘플:", tokenized_train['input_ids'][0][:5])

# 위의 결과를 디코딩하여 출력
print("\n   디코딩된 학습 데이터 샘플:", tokenizer.decode(tokenized_train['input_ids'][0][:5]))

학습 데이터 인풋 크기: 20000
검증 데이터 인풋 크기: 5000
테스트 데이터 인풋 크기: 25000

토크나이징된 학습 데이터 샘플: tensor([  101,  2004,  1999, 25285,  1010])

   디코딩된 학습 데이터 샘플: [CLS] as in amelie,


지금까지 정말 수고 많으셨습니다 🤗 지난 주 보다는 조금 어려웠는데 잘 따라오셨나요?

이전에 작성한 코드를 바탕으로 Dataset을 상속하는 IMDbDataset 클래스를 만들어 볼게요. 이 클래스는 IMDB 데이터셋을 효율적으로 처리하고, 다음 주에 진행할 언어 모델 파인튜닝에 사용될 거예요.

In [9]:
import torch
from datasets import Dataset

class IMDbDataset(Dataset):
    def __init__(self, tokenized_data, labels):
        self.input_ids = tokenized_data['input_ids'].clone().detach()
        self.attention_mask = tokenized_data['attention_mask'].clone().detach()
        self.labels = torch.tensor(labels)

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return {
            'input_ids': self.input_ids[idx],
            'attention_mask': self.attention_mask[idx],
            'labels': self.labels[idx]
        }

# 원본 데이터셋에서 labels 추출
train_labels = train_dataset['label']
val_labels = valid_dataset['label']
test_labels = test_dataset['label']

# 클래스 인스턴스화
train_dataset = IMDbDataset(tokenized_train, train_labels)
val_dataset = IMDbDataset(tokenized_val, val_labels)
test_dataset = IMDbDataset(tokenized_test, test_labels)

# 결과 확인
print("학습 데이터 크기:", len(train_dataset))
print("검증 데이터 크기:", len(val_dataset))
print("테스트 데이터 크기:", len(test_dataset))

# 데이터 샘플 확인
print("\n학습 데이터 샘플:")
sample = train_dataset[0]
print("Input IDs:", sample['input_ids'][:10], "...")  # 처음 10개 토큰만 출력
print("Attention Mask:", sample['attention_mask'][:10], "...")
print("Label:", sample['labels'])

학습 데이터 크기: 20000
검증 데이터 크기: 5000
테스트 데이터 크기: 25000

학습 데이터 샘플:
Input IDs: tensor([  101,  2004,  1999, 25285,  1010,  3522,  2413,  3152,  4025,  2000]) ...
Attention Mask: tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1]) ...
Label: tensor(1)


> 📝 위 코드를 이해하기 위한 개념들을 아래에다 정리해보았어요.
- `텐서(Tensor)`
  - 숫자 데이터를 담는 다차원 배열입니다.
  - 머신러닝과 딥러닝에서 데이터를 표현하는 기본 단위입니다.
  - NumPy의 배열과 유사하지만, GPU에서 빠른 연산이 가능합니다.
  - PyTorch에서는 torch.tensor()로 생성합니다.
- `input_ids`
  - 텍스트를 숫자로 변환한 결과입니다.
  - 각 단어나 부분 단어(서브워드)가 고유한 정수 ID로 표현됩니다.
  - 예: "I love NLP" → [101, 1045, 2293, 19204, 102]
  - 101과 102는 각각 문장의 시작과 끝을 나타내는 특수 토큰입니다.
- `attention_mask`
  - 실제 입력 데이터와 패딩을 구분하는 이진 마스크입니다.
  - 1은 실제 토큰을, 0은 패딩 토큰을 나타냅니다.
  - 모델이 패딩된 부분을 무시하고 실제 입력에만 주목하게 합니다.
  - 예: [1, 1, 1, 1, 1, 0, 0] (5개의 실제 토큰, 2개의 패딩)
- `labels`
  - 각 텍스트 샘플의 분류 결과를 나타내는 숫자입니다.
  - 이진 분류의 경우, 보통 0과 1로 표현합니다.
  - IMDB 데이터셋의 경우: 0은 부정적 리뷰, 1은 긍정적 리뷰
  - 다중 분류 작업에서는 클래스 수만큼의 고유한 정수를 사용합니다.